# Laboratory work #4

Import all significant libraries for this project.

In [1]:
# Import TensorFlow & Keras Libraries
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Input
from keras.layers import Dropout, BatchNormalization
from keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from keras.optimizers import Adam, RMSprop

# Import scikit-lean libraries
from sklearn.model_selection import train_test_split

# Import other libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib

## Exercise 1

#### Downloading Dataset
Our dataset is  (from [kaggle]())

### Preprocessing Dataset